<a href="https://colab.research.google.com/github/avaneeshreddy09-create/GDP-Data-Extraction-and-Processing/blob/main/GDP_Data_Extraction_and_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

base_url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"
response = requests.get(base_url)
soup = BeautifulSoup(response.text, "html.parser")

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

URL="https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"

# Add a User-Agent header to mimic a web browser and avoid 403 Forbidden error
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Make the request with the headers
response = requests.get(URL, headers=headers)

# Check if the request was successful
response.raise_for_status()

tables = pd.read_html(response.text)
df = tables[2]

# keep Country + IMF GDP column
df = df[['Country/Territory', 'IMF (2025)[6]']]

# clean country names (remove footnote markers)
df['Country/Territory'] = df['Country/Territory'].astype(str).str.replace(r'\[.*?\]', '', regex=True)

# clean GDP values
df['IMF (2025)[6]'] = df['IMF (2025)[6]'].astype(str)
df['IMF (2025)[6]'] = df['IMF (2025)[6]'].str.replace(r'\[.*?\]', '', regex=True)   # remove footnotes
df['IMF (2025)[6]'] = df['IMF (2025)[6]'].str.replace(',', '')                      # remove commas
df['IMF (2025)[6]'] = pd.to_numeric(df['IMF (2025)[6]'], errors='coerce')

# Convert to billions
df['GDP (Billion USD)'] = (df['IMF (2025)[6]'] / 1000).round(2)

# Keep only final columns
df = df[['Country/Territory', 'GDP (Billion USD)']]
df.columns = ['Country', 'GDP (Billion USD)']

# top 10
df = df.iloc[0:11, :]

df.to_csv('./Largest_economies.csv', index=False)

df_loaded = pd.read_csv('./Largest_economies.csv')
print(df_loaded)



           Country  GDP (Billion USD)
0            World          117165.39
1    United States           30615.74
2            China           19398.58
3          Germany            5013.57
4            Japan            4279.83
5            India            4125.21
6   United Kingdom            3958.78
7           France            3361.56
8            Italy            2543.68
9           Russia            2540.66
10          Canada            2283.60
